In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

%matplotlib inline
warnings.filterwarnings('ignore')

#  Cancer  Data Set  

### Breast Cancer Wisconsin (Diagnostic) Data Set

**Abstract:** Diagnostic Wisconsin Breast Cancer Database
Source:

Creators:

1. Dr. William H. Wolberg, General Surgery Dept.
University of Wisconsin, Clinical Sciences Center
Madison, WI 53792
wolberg '@' eagle.surgery.wisc.edu

2. W. Nick Street, Computer Sciences Dept.
University of Wisconsin, 1210 West Dayton St., Madison, WI 53706
street '@' cs.wisc.edu 608-262-6619

3. Olvi L. Mangasarian, Computer Sciences Dept.
University of Wisconsin, 1210 West Dayton St., Madison, WI 53706
olvi '@' cs.wisc.edu

### Información del conjunto de datos:

Las características se calculan a partir de una imagen digitalizada de aspiración con aguja fina (PAAF) de una masas mamarias. Describen las características de los núcleos celulares presentes en la imagen (más detalles http://pages.cs.wisc.edu/~street/images/)

Attribute Information:

1. ID number
2. Diagnosis (M = malignant, B = benign) 

**Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter)

b) texture (standard deviation of gray-scale values)

c) perimeter

d) area

e) smoothness (local variation in radius lengths)

f) compactness (perimeter^2 / area - 1.0)

g) concavity (severity of concave portions of the contour)

h) concave points (number of concave portions of the contour)

i) symmetry

j) fractal dimension ("coastline approximation" - 1)

In [ ]:
# Cargamos los datos del fichero CSV
cancer_df = pd.read_csv('../../data/06_breast-cancer-wisconsin-data.csv')

# Imprimimos los datos cargados con pandas
#df.describe()
#df = df.set_index('id')
cancer_df.info()

## Preparamos Datos

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(cancer_df['diagnosis'])
cancer_df['diagnosis_cod'] = le.transform(cancer_df['diagnosis'])
cancer_df = cancer_df.drop(['Unnamed: 32','id','diagnosis'], axis=1)
cancer_df
#le.inverse_transform(iris_df.species_cod)

In [ ]:
# separamos datos en data y target
cancer_data = cancer_df.drop(['diagnosis_cod'], axis=1)
cancer_target = cancer_df.diagnosis_cod


In [ ]:
# Separamos los Datos.... Entrenamiento y test
#?  train_test_split()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer_data, cancer_target,
                                                    test_size=0.33,
                                                    random_state=None,
                                                    shuffle =None)

print('Set de datos para Entrenamiento =',len(X_train))
print('Set de datos para Test',len(X_test))
print('Total',len(X_test)+len(X_train))
print('Data Shape=',X_test.shape)
print('Target Shape =',y_test.shape)

X_train.head()
#X_train.columns

#  Árbol de Decisión Cancer Data Set 

In [ ]:
## Modelo
arbol = DecisionTreeClassifier(max_depth=None, criterion='entropy', random_state=4)

#? DecisionTreeClassifier

In [ ]:
# ? cross_val_score()
cross_val_score(arbol, cancer_data, cancer_target)
arbol.fit(X_train,y_train)

print ("Score with data Tes",arbol.score(X_test,y_test))
print ("Score with data Train",arbol.score(X_train,y_train))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

cancer_target_names = ['M','B']
cancer_feature_names =['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']



export_graphviz(arbol,out_file='arbol.dot',
                class_names=cancer_target_names,
                feature_names=cancer_feature_names,
                impurity=False,
                filled=True,
                rounded=False,
                max_depth=None,
                rotate=False,
               )

with open('arbol.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

!dot -Tpng arbol.dot -o arbol.png
Image('arbol.png', width=2e8)


In [ ]:
cara = cancer_data.shape[1]
plt.figure(figsize=(25, 10))
plt.barh(range(cara),arbol.feature_importances_)
plt.yticks(np.arange(cara),cancer_feature_names,fontsize=18)
plt.xlabel('Importancia de las Caracteristicas',fontsize=28)
plt.ylabel('Caracteristicas',fontsize=20)
plt.xticks(fontsize=18)
plt.show()

#  Alguna Predicción 

In [ ]:
# Alguna Predicción....
warnings.filterwarnings('ignore')
ind = 413
a = cancer_data.iloc[ind]
b = cancer_target.iloc[ind]
new_x = a.values
print(a)
print('\ndiagnosis =>',b,le.inverse_transform([b]))


prediccion = arbol.predict([new_x])
#prediccion = arbol.predict(X_test)
print('\npredicción =>',prediccion, le.inverse_transform(prediccion))

#  Random Forests  Cancer Data Set 

In [ ]:
# ---------------- Random Forests
#X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

forest = RandomForestClassifier(n_estimators=1000, random_state=0)
forest.fit(X_train,y_train)

#you can tune parameter such as:
# - n_job (how many cores)(n_job=-1 => all cores)
# - max_depth
# - max_feature


print('acc for training data: {:.2f}'.format(forest.score(X_train,y_train)))
print('acc for test data: {:.2f}'.format(forest.score(X_test,y_test)))


# =========================================================================================
warnings.filterwarnings('ignore')
# Alguna Predicción....
ind = 496
a = cancer_data.iloc[ind]
b = cancer_target.iloc[ind]
new_x = a.values
print(a)
print('\ndiagnosis =>',b,le.inverse_transform([b]))


prediccion = forest.predict([new_x])
#prediccion = arbol.predict(X_test)
print('\npredicción =>',prediccion, le.inverse_transform(prediccion))


#   Diabetes Data Set 


Michael Kahn, MD, PhD, Universidad de Washington, St. Louis, MO

### Información del conjunto de datos:

Los registros de pacientes con diabetes se obtuvieron de dos fuentes: a) un dispositivo de registro electrónico automático y b) registros en papel. 

El dispositivo automático tenía un reloj interno para los eventos de marca de tiempo, mientras que los registros en papel solo proporcionaban espacios de "tiempo lógico" (desayuno, almuerzo, cena, hora de acostarse). 

Para los registros en papel, se asignaron horarios fijos para el desayuno (08:00), el almuerzo (12:00), la cena (18:00) y la hora de acostarse (22:00). Por lo tanto, los registros en papel tienen tiempos de grabación uniformes ficticios, mientras que los registros electrónicos tienen marcas de tiempo más realistas.

Los archivos de diabetes constan de cuatro campos por registro. Cada campo está separado por una pestaña y cada registro está separado por una nueva línea.

Nombres de archivos y formato

(1) Fecha en formato MM-DD-YYYY
(2) Tiempo en formato XX: YY
(3) Código
(4) Valor

El campo Código se descifra de la siguiente manera:

33 = Dosis de insulina regular

34 = dosis de insulina NPH

35 = dosis de insulina UltraLente

48 = medición de glucosa en sangre no especificada

57 = medición de glucosa en sangre no especificada

58 = medición de glucosa en sangre antes del desayuno

59 = medición de glucosa en sangre después del desayuno

60 = medición de glucosa en sangre antes del almuerzo

61 = medición de glucosa en sangre después del almuerzo

62 = medición de glucosa en sangre antes de la cena

63 = medición de glucosa en sangre después de la cena

64 = Medición de glucosa en sangre antes del aperitivo

65 = Síntomas de hipoglucemia

66 = Ingestión típica de comida

67 = Ingestión de comida más de lo habitual

68 = ingesta de comida menos de lo usual

69 = actividad típica de ejercicio

70 = Actividad de ejercicio más de lo habitual

71 = actividad de ejercicio menos de lo 


In [ ]:
# Cargamos los datos del fichero CSV
diabetes_df = pd.read_csv('../../data/06_diabetes.csv')

# Imprimimos los datos cargados con pandas
#diabetes_df.info()
diabetes_df

In [ ]:
# separamos datos en data y target
diabetes_data   = diabetes_df.drop(['diagnosis'], axis=1)
diabetes_target = diabetes_df.diagnosis

In [ ]:
# Separamos los Datos.... Entrenamiento y test
#?  train_test_split()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(diabetes_data, diabetes_target,
                                                    test_size=0.33,
                                                    random_state=None,
                                                    shuffle =None)

print('Set de datos para Entrenamiento =',len(X_train))
print('Set de datos para Test',len(X_test))
print('Total',len(X_test)+len(X_train))
print('Data Shape=',X_test.shape)
print('Target Shape =',y_test.shape)

X_train.head()
#X_train.columns

In [ ]:
## Modelo
arbol = DecisionTreeClassifier(max_depth=None, criterion='entropy', random_state=4)
#arbol = DecisionTreeClassifier()

#? DecisionTreeClassifier

# ? cross_val_score()
cross_val_score(arbol, diabetes_data, diabetes_target)
arbol.fit(X_train,y_train)

print ("Score with data Tes",arbol.score(X_test,y_test))
print ("Score with data Train",arbol.score(X_train,y_train))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

diabetes_target_names = ['0','1']
diabetes_feature_names = X_train.columns



export_graphviz(arbol,out_file='arbol.dot',
                class_names=diabetes_target_names,
                impurity=False,
                filled=True,
                rounded=False,
                max_depth=None,
                rotate=False,
                feature_names=diabetes_feature_names,
               )

with open('arbol.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)



!dot -Tpng arbol.dot -o arbol.png
Image('arbol.png', width=4e8)


In [ ]:
cara = diabetes_data.shape[1]
plt.figure(figsize=(25, 10))
plt.barh(range(cara),arbol.feature_importances_)
plt.yticks(np.arange(cara),cancer_feature_names,fontsize=18)
plt.xlabel('Importancia de las Caracteristicas',fontsize=28)
plt.ylabel('Caracteristicas',fontsize=20)
plt.xticks(fontsize=18)
plt.show()

In [ ]:
warnings.filterwarnings('ignore')
# Alguna Predicción....

ind = 58
a = diabetes_data.iloc[ind]
b = diabetes_target.loc[ind]
new_x = a.values
print(a)
print('\ndiagnosis =>',b)


prediccion = arbol.predict([new_x])
#prediccion = arbol.predict(X_test)
print('\npredicción =>',prediccion)

#  Random Forests Diabetes Data Set 

In [ ]:
# ---------------- Random Forests
#X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

forest = RandomForestClassifier(n_estimators=1000, random_state=0)
forest.fit(X_train,y_train)

#you can tune parameter such as:
# - n_job (how many cores)(n_job=-1 => all cores)
# - max_depth
# - max_feature


print('acc for training data: {:.2f}'.format(forest.score(X_train,y_train)))
print('acc for test data: {:.2f}'.format(forest.score(X_test,y_test)))


In [ ]:
warnings.filterwarnings('ignore')
# Alguna Predicción....

ind = 13
a = diabetes_data.iloc[ind]
b = diabetes_target.iloc[ind]
new_x = a.values
print(a)
print('\ndiagnosis =>',b)


prediccion = forest.predict([new_x])
#prediccion = arbol.predict(X_test)
print('\npredicción =>',prediccion)

In [ ]:
p =forest.predict(X_test)


print ('Accuracy:', accuracy_score(y_test, p))
print ('\nConfusion Matrix:\n', confusion_matrix(y_test, p))
print ('\nClassification Report:', classification_report(y_test, p))
